# Automated ML

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [1]:
from azureml.core import Workspace, Experiment
from azureml.core import Model 
from azureml.train.automl import AutoMLConfig
from azureml.widgets import RunDetails
from azureml.core.compute import ComputeTarget, AmlCompute
from pprint import pprint 
import joblib
import os

## Dataset

### Overview
TODO: In this markdown cell, give an overview of the dataset you are using. Also mention the task you will be performing.


TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [2]:
#getting data: the dataset was uploaded to and is accesible through my github repo
from azureml.data.dataset_factory import TabularDatasetFactory

rawdata_homic2018 = "https://raw.githubusercontent.com/hftamayo/azuremlproj03/main/cad2018.csv"
dshomic2018 = TabularDatasetFactory.from_delimited_files(path=rawdata_homic2018, separator=',')
pdhomic2018 = dshomic2018.to_pandas_dataframe().dropna()

In [3]:
#setting up workspace and experiment
ws = Workspace.from_config()

# choose a name for experiment
experiment_name = 'expp03ht'

experiment=Experiment(ws, experiment_name)

In [4]:
# setting up IaaS where the experiment will run on it
ccname = "ccp03aml"
try:
    ccluster = ComputeTarget(ws, ccname)
    print(f"{ccname} exists or is in use!, choose a different name")
except:
    ccconfig = AmlCompute.provisioning_configuration(vm_size="Standard_D2_V2", max_nodes=5)
    ccluster = ComputeTarget.create(ws, ccname, ccconfig)
ccluster.wait_for_completion(show_output=True)

Creating
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


## AutoML Configuration

TODO: Explain why you chose the automl settings and cofiguration you used below.

In [5]:
# TODO: Put your automl settings here
automl_settings = {
    "experiment_timeout_minutes": 20,
    "max_concurrent_iterations": 5,
    "primary_metric" : 'accuracy'
}

# TODO: Put your automl config here
automl_config = AutoMLConfig(
        task='classification',
        compute_target=ccluster,
        training_data=dshomic2018,
        label_column_name='sexo',
        n_cross_validations=5,
        **automl_settings
)

In [19]:
# TODO: Submit your experiment
exprun = experiment.submit(automl_config)

Running on remote.


## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [20]:
RunDetails(exprun).show()
exprun.wait_for_completion(show_output=True)

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…


Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetBalancing. Performing class balancing sweeping
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.
Current status: ModelSelection. Beginning model selection.

****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       ALERTED
DESCRIPTION:  To decrease model bias, please cancel the current run and fix balancing problem.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData
DETAILS:      Imbalanced data can lead to a falsely perceived positive effect of a model's accuracy because the input data has bias towards one class.
+---------------------------------+---------------------------------+--------------------------------------+
|Size of the smallest class       |Name/Label of the smallest class |Number of 

        60   StandardScalerWrapper XGBoostClassifier        0:00:56       0.8790    0.8813
        61   SparseNormalizer LightGBM                      0:00:54       0.8790    0.8813
        62   SparseNormalizer XGBoostClassifier             0:00:53       0.8793    0.8813
        63   StandardScalerWrapper XGBoostClassifier        0:00:48       0.8802    0.8813
        64   MaxAbsScaler LightGBM                          0:00:49       0.8790    0.8813
        65   StandardScalerWrapper XGBoostClassifier        0:00:53       0.8790    0.8813
        66   SparseNormalizer XGBoostClassifier             0:00:49       0.8790    0.8813
        67   StandardScalerWrapper LightGBM                 0:00:59       0.8790    0.8813
        68   StandardScalerWrapper RandomForest             0:00:52       0.8790    0.8813
        69   MaxAbsScaler LightGBM                          0:00:45       0.8790    0.8813
        71   SparseNormalizer LightGBM                      0:00:45       0.8790    0.8813

{'runId': 'AutoML_9038d9fb-0e41-4e62-95b0-a349f7ddbda5',
 'target': 'ccp03aml',
 'status': 'Completed',
 'startTimeUtc': '2021-01-29T12:15:59.253016Z',
 'endTimeUtc': '2021-01-29T12:47:19.976252Z',
 'properties': {'num_iterations': '1000',
  'training_type': 'TrainFull',
  'acquisition_function': 'EI',
  'primary_metric': 'accuracy',
  'train_split': '0',
  'acquisition_parameter': '0',
  'num_cross_validation': '5',
  'target': 'ccp03aml',
  'AMLSettingsJsonString': '{"path":null,"name":"expp03ht","subscription_id":"1b944a9b-fdae-4f97-aeb1-b7eea0beac53","resource_group":"aml-quickstarts-136496","workspace_name":"quick-starts-ws-136496","region":"southcentralus","compute_target":"ccp03aml","spark_service":null,"azure_service":"remote","many_models":false,"pipeline_fetch_max_batch_size":1,"iterations":1000,"primary_metric":"accuracy","task_type":"classification","data_script":null,"validation_size":0.0,"n_cross_validations":5,"y_min":null,"y_max":null,"num_classes":null,"featurization":

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [21]:
#import azureml.train.automl
# obtaining best run and fitted model
brmodel, fmodel = exprun.get_output()

# Print the best run
print(brmodel)

# Get all metrics of the best run model
brmetrics = brmodel.get_metrics()

# Print all metrics of the best run model
for metric_name in brmetrics:
    metric = brmetrics[metric_name]
    print(metric_name, metric)
    

Run(Experiment: expp03ht,
Id: AutoML_9038d9fb-0e41-4e62-95b0-a349f7ddbda5_80,
Type: azureml.scriptrun,
Status: Completed)
precision_score_micro 0.8819476607991433
recall_score_macro 0.5147719715624961
f1_score_weighted 0.8304641276084948
accuracy 0.8819476607991433
weighted_accuracy 0.9812271957797485
AUC_weighted 0.6187331807428572
log_loss 0.3631963089285146
recall_score_weighted 0.8819476607991433
balanced_accuracy 0.5147719715624961
matthews_correlation 0.1310981942998046
AUC_macro 0.6187331807428572
precision_score_macro 0.7876990337092691
average_precision_score_micro 0.8850169345226563
f1_score_macro 0.4973371657510099
f1_score_micro 0.8819476607991433
average_precision_score_macro 0.567234045526078
precision_score_weighted 0.8583053096012531
recall_score_micro 0.8819476607991433
norm_macro_recall 0.029543943124992377
AUC_micro 0.9041245675687868
average_precision_score_weighted 0.8275860830972966
accuracy_table aml://artifactId/ExperimentRun/dcid.AutoML_9038d9fb-0e41-4e62-95b0-

In [22]:
# Display parameters of the fitted model
def print_model(model, prefix=""):
    for step in model.steps:
        print(prefix + step[0])
        if hasattr(step[1], 'estimators') and hasattr(step[1], 'weights'):
            pprint({'estimators': list(
                e[0] for e in step[1].estimators), 'weights': step[1].weights})
            print()
            for estimator in step[1].estimators:
                print_model(estimator[1], estimator[0] + ' - ')
        else:
            pprint(step[1].get_params())
            print()

print_model(fmodel)

datatransformer
{'enable_dnn': None,
 'enable_feature_sweeping': None,
 'feature_sweeping_config': None,
 'feature_sweeping_timeout': None,
 'featurization_config': None,
 'force_text_dnn': None,
 'is_cross_validation': None,
 'is_onnx_compatible': None,
 'logger': None,
 'observer': None,
 'task': None,
 'working_dir': None}

prefittedsoftvotingclassifier
{'estimators': ['20', '76', '63', '21', '1', '16'],
 'weights': [0.4, 0.2, 0.1, 0.1, 0.1, 0.1]}

20 - sparsenormalizer
{'copy': True, 'norm': 'max'}

20 - xgboostclassifier
{'base_score': 0.5,
 'booster': 'gbtree',
 'colsample_bylevel': 1,
 'colsample_bynode': 1,
 'colsample_bytree': 1,
 'eta': 0.2,
 'gamma': 0,
 'learning_rate': 0.1,
 'max_delta_step': 0,
 'max_depth': 6,
 'max_leaves': 15,
 'min_child_weight': 1,
 'missing': nan,
 'n_estimators': 25,
 'n_jobs': 1,
 'nthread': None,
 'objective': 'reg:logistic',
 'random_state': 0,
 'reg_alpha': 0,
 'reg_lambda': 0.4166666666666667,
 'scale_pos_weight': 1,
 'seed': None,
 'silent': 

In [23]:
import joblib
joblib.dump(value=fmodel, filename='model.pkl')

['model.pkl']

In [24]:
#TODO: Save the best model
autobmlp3ht = brmodel.register_model(model_path='outputs/model.pkl', model_name='p3automl_ht',
                        tags={'Method of execution':'Auto ML'},
                        properties={'Accuracy': brmetrics['accuracy']})

print(autobmlp3ht)

Model(workspace=Workspace.create(name='quick-starts-ws-136496', subscription_id='1b944a9b-fdae-4f97-aeb1-b7eea0beac53', resource_group='aml-quickstarts-136496'), name=p3automl_ht, id=p3automl_ht:2, version=2, tags={'Method of execution': 'Auto ML'}, properties={'Accuracy': '0.8819476607991433'})


## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [25]:
# Download scoring file 
brmodel.download_file('outputs/scoring_file_v_1_0_0.py', 'scoring.py')

# Download environment file
brmodel.download_file('outputs/conda_env_v_1_0_0.yml', 'enviro.yml')

In [26]:
from azureml.core.environment import Environment
from azureml.core.model import InferenceConfig

infcfg = InferenceConfig(entry_script='scoring.py',
                                    environment=Environment.from_conda_specification(name='myenv',file_path='enviro.yml'))

# deploying the model via WebService
from azureml.core.webservice import AciWebservice

depcfg = AciWebservice.deploy_configuration(cpu_cores = 1, memory_gb = 1)
webservice = Model.deploy(ws, "wsp03ht01", [autobmlp3ht], infcfg, depcfg)
webservice.wait_for_deployment(show_output = True)
print(webservice.state)

print(webservice.scoring_uri)

print(webservice.swagger_uri)

Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running.............................................
Succeeded
ACI service creation operation finished, operation "Succeeded"
Healthy
http://94e18276-4983-4b4f-925f-845d23fa7c95.southcentralus.azurecontainer.io/score
http://94e18276-4983-4b4f-925f-845d23fa7c95.southcentralus.azurecontainer.io/swagger.json


In [35]:
import json

#Importing the data set for testing 
tsthomic2018 = pdhomic2018.sample(10) 
lblhomic2018 = tsthomic2018.pop('sexo')

tstdatahomic2018 = json.dumps({'data': tsthomic2018.to_dict(orient='records')})

print(tstdatahomic2018)

{"data": [{"id": 3053, "regfalle": 24117, "edad": 25, "deptoocuhe": 12, "tipoarma": 1, "pracaut": 1}, {"id": 3291, "regfalle": 23895, "edad": 25, "deptoocuhe": 11, "tipoarma": 1, "pracaut": 1}, {"id": 2855, "regfalle": 23586, "edad": 52, "deptoocuhe": 3, "tipoarma": 1, "pracaut": 1}, {"id": 2478, "regfalle": 21574, "edad": 56, "deptoocuhe": 6, "tipoarma": 1, "pracaut": 1}, {"id": 3170, "regfalle": 22237, "edad": 31, "deptoocuhe": 6, "tipoarma": 1, "pracaut": 0}, {"id": 3264, "regfalle": 22189, "edad": 29, "deptoocuhe": 8, "tipoarma": 1, "pracaut": 0}, {"id": 2144, "regfalle": 20845, "edad": 27, "deptoocuhe": 3, "tipoarma": 13, "pracaut": 1}, {"id": 1086, "regfalle": 21339, "edad": 25, "deptoocuhe": 10, "tipoarma": 1, "pracaut": 1}, {"id": 2613, "regfalle": 23573, "edad": 17, "deptoocuhe": 3, "tipoarma": 9, "pracaut": 1}, {"id": 1652, "regfalle": 23416, "edad": 34, "deptoocuhe": 7, "tipoarma": 13, "pracaut": 1}]}


TODO: In the cell below, send a request to the web service you deployed to test it.

In [36]:
import requests

headers = {'Content-type': 'application/json'}

response = requests.post(webservice.scoring_uri, tstdatahomic2018, headers=headers)

In [37]:
# Print results 
print(response.text)

"{\"result\": [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]}"


In [34]:
# Print original labels
print(lblhomic2018)

1465    0
2849    0
1726    0
3190    0
2201    1
1940    0
1142    0
1557    0
801     0
3002    0
Name: sexo, dtype: int64


TODO: In the cell below, print the logs of the web service and delete the service

In [31]:
print(webservice.get_logs())

/usr/sbin/nginx: /azureml-envs/azureml_4f26b5b9300bb3a3b89415c34dafac63/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_4f26b5b9300bb3a3b89415c34dafac63/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_4f26b5b9300bb3a3b89415c34dafac63/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_4f26b5b9300bb3a3b89415c34dafac63/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_4f26b5b9300bb3a3b89415c34dafac63/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
2021-01-29T12:55:48,207323048+00:00 - iot-server/run 
2021-01-29T12:55:48,207505750+00:00 - nginx/run 
2021-01-29T12:55:48,209240467+00:00 - rsyslog/run 
rsyslogd: /azureml-envs/azureml_4f26b5b9300bb3a3b89415c34daf

In [38]:
#best practice: deleting webservice that it won't be used anymore
webservice.delete()

In [39]:
#best practice: deleting CCluster that it won't be used anymore
ccluster.delete()